In [2]:
%run gw_exp.ipynb

ModuleNotFoundError: No module named 'ipynb'

In [ ]:
# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - number of boundary marked points, a non-negative integer
def ogw_exp(n, A, b, k):

    A.sort()    # asceding order
    A.reverse() # to descending order

    for c_d in A:
        if c_d > n:
            return 0

    # wall-crossing theorem (Elad)
    if A[-1] == -1:
        del A[-1] # remove the last element
        return -ogw_exp(n, A, b, k+1)

    #degree axiom (Elad)
    if 2 * sum(A)!= (n - 3 + (n+1)*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        return 0

    # unit axiom (May)

    # divisor axiom (May) 


    t = ogw_exp(n, [A[0]-1, A[1] + 2] + A[2:], b, k)
    